In [169]:
# ! conda install -c conda-forge imbalanced-learn
# ! pip install nose
# ! pip install imbalanced-ensemble           
! pip install threadpoolctl==3.1.0

  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 2.2.0
    Uninstalling threadpoolctl-2.2.0:
      Successfully uninstalled threadpoolctl-2.2.0


In [11]:
import re

def read_dot_dat_file(path):
    datContent = [i.strip().split() for i in open(path).readlines()]
    r = re.compile("@inputs.*")
    _at_data = datContent.index(['@data'])
    assert datContent[0][0] == '@relation'
    assert datContent[_at_data-1][0] == '@outputs'
    assert datContent[_at_data-2][0] == '@inputs'
    print(datContent[_at_data-3][2:])
    assert len(datContent[_at_data-3][2:]) == 2   # Two Class

    col_names = datContent[_at_data-2][1:]
    col_names.append(datContent[_at_data-1][1])
    
    df = pd.read_csv(path, skiprows=_at_data+1, names=col_names, sep=r', ', engine='python')
    # df = pd.read_csv(path, skiprows=_at_data+1, names=col_names, sep=", ", engine='python')

    class1 = datContent[_at_data-3][2:][0].replace("{","").replace(",","")
    class2 = datContent[_at_data-3][2:][1].replace("}","").replace(",","")

    df['Class'] = df['Class'].replace({class1: 1, class2: -1})
    
    return df



# temp

In [121]:
import pandas as pd 
import numpy as np
import os

base_path = 'Datasets/'
# need_to_convert = ['wisconsin-5-fold','new-thyroid2-5-fold','new-thyroid1-5-fold']
# need_to_convert = ['yeast-1_vs_7-5-fold', 'led7digit-0-2-4-5-6-7-8-9_vs_1-5-fold', 'ecoli-0-3-4-6_vs_5-5-fold', 'abalone19-5-', 'abalone19-5-fold']
need_to_convert = ['ecoli-0-1_vs_5-5-fold','ecoli-0-1-4-7_vs_5-6-5-fold','ecoli-0-3-4-7_vs_5-6-5-fold','glass-0-1-4-6_vs_2-5-fold', 'ecoli-0-4-6_vs_5-5-fold',
                  'yeast-0-3-5-9_vs_7-8-5-fold','yeast-1-2-8-9_vs_7-5-fold']
for needed in need_to_convert[5:]:
    for datFile in glob.glob(base_path+needed+'/*.dat'):
        print(datFile)
        df = read_dot_dat_file(datFile)
        df.to_excel(base_path+needed+'/'+datFile.split('/')[-1].split('.')[0]+'.xlsx')


Datasets/yeast-0-3-5-9_vs_7-8-5-fold/yeast-0-3-5-9_vs_7-8-5-2tst.dat
['{1,', '-1}']
Datasets/yeast-0-3-5-9_vs_7-8-5-fold/yeast-0-3-5-9_vs_7-8-5-1tst.dat
['{1,', '-1}']
Datasets/yeast-0-3-5-9_vs_7-8-5-fold/yeast-0-3-5-9_vs_7-8-5-4tra.dat
['{1,', '-1}']
Datasets/yeast-0-3-5-9_vs_7-8-5-fold/yeast-0-3-5-9_vs_7-8-5-4tst.dat
['{1,', '-1}']
Datasets/yeast-0-3-5-9_vs_7-8-5-fold/yeast-0-3-5-9_vs_7-8-5-5tst.dat
['{1,', '-1}']
Datasets/yeast-0-3-5-9_vs_7-8-5-fold/yeast-0-3-5-9_vs_7-8-5-5tra.dat
['{1,', '-1}']
Datasets/yeast-0-3-5-9_vs_7-8-5-fold/yeast-0-3-5-9_vs_7-8-5-3tra.dat
['{1,', '-1}']
Datasets/yeast-0-3-5-9_vs_7-8-5-fold/yeast-0-3-5-9_vs_7-8-5-2tra.dat
['{1,', '-1}']
Datasets/yeast-0-3-5-9_vs_7-8-5-fold/yeast-0-3-5-9_vs_7-8-5-3tst.dat
['{1,', '-1}']
Datasets/yeast-0-3-5-9_vs_7-8-5-fold/yeast-0-3-5-9_vs_7-8-5-1tra.dat
['{1,', '-1}']
Datasets/yeast-1-2-8-9_vs_7-5-fold/yeast-1-2-8-9_vs_7-5-1tst.dat
['{1,', '-1}']
Datasets/yeast-1-2-8-9_vs_7-5-fold/yeast-1-2-8-9_vs_7-5-2tra.dat
['{1,', '-1}']


In [12]:
import time
import os
import glob
import numpy as np
import random  
import pandas as pd 
import random

from sklearn.tree import DecisionTreeClassifier 

from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc,f1_score,matthews_corrcoef
from imblearn.metrics import geometric_mean_score


from imblearn.ensemble import RUSBoostClassifier
from imbalanced_ensemble.ensemble import SMOTEBoostClassifier
from maatpy.classifiers import AdaCost


In [165]:
base_path = 'Datasets/'
datasets  = os.listdir(base_path) 
classifiers = {"RUS": RUSBoostClassifier(random_state=0, algorithm='SAMME', estimator=DecisionTreeClassifier(max_depth=10)),
              "SMOTE": SMOTEBoostClassifier(estimator=DecisionTreeClassifier(max_depth=10), n_estimators = 100, algorithm='SAMME', random_state=0),
              "Ada1": AdaCost(base_estimator=DecisionTreeClassifier(max_depth=10), n_estimators=100, algorithm='adac1',random_state=0),
              "AdaCost": AdaCost(base_estimator=DecisionTreeClassifier(max_depth=10), n_estimators=100, algorithm='adacost',random_state=0),}


def clasify(dataset, classifier):
    mcc = []
    f1 = []
    auc_a = []
    gmean = []
    times = []
    y_preds = []
    traFiles = sorted(glob.glob(base_path+dataset+'/*tra.xlsx'))
    tstFiles = sorted(glob.glob(base_path+dataset+'/*tst.xlsx'))
    for traPath, tstPath in zip(traFiles, tstFiles):
        print(traPath)
        
        df_train = pd.read_excel(traPath)
        df_test = pd.read_excel(tstPath)

        x_train= df_train.iloc[:, 1:-1]
        y_train = df_train.iloc[:, -1]
        x_test= df_test.iloc[:, 1:-1]
        y_test = df_test.iloc[:, -1]
        
        #####
        # for some dataset get error Unknown label type: 'unknown'
        y_train = y_train.astype('int')
        y_test = y_test.astype('int')
        
        st = time.time()
        clf = classifiers[classifier]

        clf.fit(x_train, y_train)

        y_pred = clf.predict(x_test) 
        et = time.time()
        y_preds.append(y_pred)
        # compute error
        mcc.append(matthews_corrcoef(y_test, y_pred))
        #--------------------------------
        fpr, tpr, thresholds = roc_curve(y_test, y_pred)
        auc_a.append(auc(fpr, tpr))
        #--------------------------------
        f1.append(f1_score(y_test, y_pred))
        #--------------------------------
        gmean.append(geometric_mean_score(y_test, y_pred, labels=[1, -1]))
        
        #time of train and test
        times.append(et - st)
        
    return {"mcc": mcc, "auc": auc_a, "f1": f1, "gmean": gmean, "exe_time": times, "y_pred": y_pred}

aaaaaa =  clasify(datasets[0], 'Ada1')
aaaaaa

Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-1tra.xlsx
Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-2tra.xlsx
Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-3tra.xlsx
Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-4tra.xlsx
Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-5tra.xlsx


{'mcc': [-0.056313829183370835,
  -0.056313829183370835,
  0.19936606554975947,
  0.23095609134944634,
  0.7132259743375],
 'auc': [0.47674418604651164,
  0.47674418604651164,
  0.5717054263565892,
  0.6317829457364341,
  0.8274509803921568],
 'f1': [0.0, 0.0, 0.2222222222222222, 0.28571428571428575, 0.7272727272727272],
 'gmean': [0.0,
  0.0,
  0.4034732923929645,
  0.5568460463897045,
  0.8116794499134278],
 'exe_time': [0.022873401641845703,
  0.029998064041137695,
  0.014106273651123047,
  0.024137020111083984,
  0.014642000198364258],
 'y_pred': array([-1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1,  1,  1, -1,  1,  1])}

In [89]:
# #temp 
for i in range(len(datasets)):
    if not os.path.isdir('./Results/'+datasets[i]):
        os.mkdir('./Results/'+datasets[i])
    for classifier in classifiers:
        if not os.path.isdir('./Results/'+datasets[i]+'/'+classifier):
            os.mkdir('./Results/'+datasets[i]+'/'+classifier)
            


In [90]:
def res_to_files(dataset, classifier, dict_res):
    np.save('./Results/'+dataset+'/'+classifier+'/mcc.npy',dict_res['mcc'])
    np.save('./Results/'+dataset+'/'+classifier+'/auc.npy',dict_res['auc'])
    np.save('./Results/'+dataset+'/'+classifier+'/f1.npy',dict_res['f1'])
    np.save('./Results/'+dataset+'/'+classifier+'/gmean.npy',dict_res['gmean'])
    np.save('./Results/'+dataset+'/'+classifier+'/exe_time.npy',dict_res['exe_time'])
    np.save('./Results/'+dataset+'/'+classifier+'/y_pred.npy',dict_res['y_pred'])
    

In [129]:
datasets

['yeast-1_vs_7-5-fold',
 'led7digit-0-2-4-5-6-7-8-9_vs_1-5-fold',
 'ecoli-0-6-7_vs_3-5-5-fold',
 'glass-0-4_vs_5-5-fold',
 'ecoli-0-2-6-7_vs_3-5-5-fold',
 'yeast-0-2-5-6_vs_3-7-8-9-5-fold',
 'yeast-0-5-6-7-9_vs_4-5-fold',
 'ecoli-0-3-4-6_vs_5-5-fold',
 'yeast5-5-fold',
 'yeast-2_vs_4-5-fold',
 'ecoli-0-1-4-6_vs_5-5-fold',
 'shuttle-c2-vs-c4-5-fold',
 'ecoli-0-1_vs_5-5-fold',
 'yeast4-5-fold',
 'abalone19-5-',
 'glass-0-1-6_vs_5-5-fold',
 'ecoli-0-1-4-7_vs_2-3-5-6-5-fold',
 'ecoli-0-1-4-7_vs_5-6-5-fold',
 'ecoli-0-3-4-7_vs_5-6-5-fold',
 'yeast-2_vs_8-5-fold',
 'ecoli-0-1-3-7_vs_2-6-5-fold',
 'yeast6-5-fold',
 'pima-5-fold',
 'glass-0-1-4-6_vs_2-5-fold',
 'abalone9-18-5-fold',
 'abalone19-5-fold',
 'glass4-5-fold',
 'ecoli-0-2-3-4_vs_5-5-fold',
 'vehicle2-5-fold',
 'glass5-5-fold',
 'ecoli-0_vs_1-5-fold',
 'ecoli-0-6-7_vs_5-5-fold',
 'glass-0-1-5_vs_2-5-fold',
 'shuttle-c0-vs-c4-5-fold',
 'ecoli-0-4-6_vs_5-5-fold',
 'new-thyroid1-5-fold',
 'ecoli4-5-fold',
 'glass-0-6_vs_5-5-fold',
 'seg

In [166]:
for classifier in classifiers:
    for dataset in datasets:
        if not os.path.exists('./Results/'+dataset+'/'+classifier+'/y_pred.npy'):
            print(classifier," => ", dataset, "=>", end=' ')
            # tmp_res = clasify(dataset, classifier)
            # print(tmp_res['auc'])
            try:
                tmp_res = clasify(dataset, classifier)
                res_to_files(dataset, classifier, tmp_res)
            except:
                print(dataset, classifier)


# classifier = 'AdaCost'
# dataset = 'abalone19-5-'
# if not os.path.exists('./Results/'+dataset+'/'+classifier+'/y_pred.npy'):
#     print(classifier," => ", dataset, "=>", end=' ')
#     tmp_res = clasify(dataset, classifier)
#     res_to_files(dataset, classifier, tmp_res)
#     print(tmp_res['auc'])

        

RUS  =>  yeast-1_vs_7-5-fold => Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-1tra.xlsx
Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-2tra.xlsx
Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-3tra.xlsx
Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-4tra.xlsx
Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-5tra.xlsx
RUS  =>  led7digit-0-2-4-5-6-7-8-9_vs_1-5-fold => Datasets/led7digit-0-2-4-5-6-7-8-9_vs_1-5-fold/led7digit-0-2-4-5-6-7-8-9_vs_1-5-1tra.xlsx
Datasets/led7digit-0-2-4-5-6-7-8-9_vs_1-5-fold/led7digit-0-2-4-5-6-7-8-9_vs_1-5-2tra.xlsx
Datasets/led7digit-0-2-4-5-6-7-8-9_vs_1-5-fold/led7digit-0-2-4-5-6-7-8-9_vs_1-5-3tra.xlsx
Datasets/led7digit-0-2-4-5-6-7-8-9_vs_1-5-fold/led7digit-0-2-4-5-6-7-8-9_vs_1-5-4tra.xlsx
Datasets/led7digit-0-2-4-5-6-7-8-9_vs_1-5-fold/led7digit-0-2-4-5-6-7-8-9_vs_1-5-5tra.xlsx
RUS  =>  ecoli-0-6-7_vs_3-5-5-fold => Datasets/ecoli-0-6-7_vs_3-5-5-fold/ecoli-0-6-7_vs_3-5-5-1tra.xlsx
Datasets/ecoli-0-6-7_vs_3-5-5-fold/ecoli-0-6-7_vs_3-5-5-2tra.xlsx
Datasets/ecoli-0-6-7_vs_3-5-

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f8224df0790>
Traceback (most recent call last):
  File "/home/ri/anaconda3/envs/dsml/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/ri/anaconda3/envs/dsml/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/ri/anaconda3/envs/dsml/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/ri/anaconda3/envs/dsml/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


SMOTE  =>  glass-0-4_vs_5-5-fold => Datasets/glass-0-4_vs_5-5-fold/glass-0-4_vs_5-5-1tra.xlsx


Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f8224df0a60>
Traceback (most recent call last):
  File "/home/ri/anaconda3/envs/dsml/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/ri/anaconda3/envs/dsml/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/ri/anaconda3/envs/dsml/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/ri/anaconda3/envs/dsml/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


Datasets/glass-0-4_vs_5-5-fold/glass-0-4_vs_5-5-2tra.xlsx


...

SMOTE  =>  glass2-5-fold => Datasets/glass2-5-fold/glass2-5-1tra.xlsx


Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f82250ec700>
Traceback (most recent call last):
  File "/home/ri/anaconda3/envs/dsml/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/ri/anaconda3/envs/dsml/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/ri/anaconda3/envs/dsml/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/ri/anaconda3/envs/dsml/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


Datasets/glass2-5-fold/glass2-5-2tra.xlsx
Datasets/glass2-5-fold/glass2-5-3tra.xlsx
Datasets/glass2-5-fold/glass2-5-4tra.xlsx
Datasets/glass2-5-fold/glass2-5-5tra.xlsx
SMOTE  =>  yeast-0-3-5-9_vs_7-8-5-fold => Datasets/yeast-0-3-5-9_vs_7-8-5-fold/yeast-0-3-5-9_vs_7-8-5-1tra.xlsx
Datasets/yeast-0-3-5-9_vs_7-8-5-fold/yeast-0-3-5-9_vs_7-8-5-2tra.xlsx
Datasets/yeast-0-3-5-9_vs_7-8-5-fold/yeast-0-3-5-9_vs_7-8-5-3tra.xlsx
Datasets/yeast-0-3-5-9_vs_7-8-5-fold/yeast-0-3-5-9_vs_7-8-5-4tra.xlsx
Datasets/yeast-0-3-5-9_vs_7-8-5-fold/yeast-0-3-5-9_vs_7-8-5-5tra.xlsx
SMOTE  =>  yeast-1-4-5-8_vs_7-5-fold => Datasets/yeast-1-4-5-8_vs_7-5-fold/yeast-1-4-5-8_vs_7-5-1tra.xlsx
Datasets/yeast-1-4-5-8_vs_7-5-fold/yeast-1-4-5-8_vs_7-5-2tra.xlsx
Datasets/yeast-1-4-5-8_vs_7-5-fold/yeast-1-4-5-8_vs_7-5-3tra.xlsx
Datasets/yeast-1-4-5-8_vs_7-5-fold/yeast-1-4-5-8_vs_7-5-4tra.xlsx
Datasets/yeast-1-4-5-8_vs_7-5-fold/yeast-1-4-5-8_vs_7-5-5tra.xlsx
SMOTE  =>  wisconsin-5-fold => Datasets/wisconsin-5-fold/wisconsin-5-1

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f82250ec8b0>
Traceback (most recent call last):
  File "/home/ri/anaconda3/envs/dsml/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/ri/anaconda3/envs/dsml/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/ri/anaconda3/envs/dsml/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/ri/anaconda3/envs/dsml/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


glass-0-1-6_vs_2-5-fold SMOTE
SMOTE  =>  yeast-1-2-8-9_vs_7-5-fold => Datasets/yeast-1-2-8-9_vs_7-5-fold/yeast-1-2-8-9_vs_7-5-1tra.xlsx
yeast-1-2-8-9_vs_7-5-fold SMOTE
SMOTE  =>  ecoli-0-3-4_vs_5-5-fold => Datasets/ecoli-0-3-4_vs_5-5-fold/ecoli-0-3-4_vs_5-5-1tra.xlsx
ecoli-0-3-4_vs_5-5-fold SMOTE
SMOTE  =>  new-thyroid2-5-fold => Datasets/new-thyroid2-5-fold/newthyroid2-5-1tra.xlsx
new-thyroid2-5-fold SMOTE
Ada1  =>  yeast-1_vs_7-5-fold => Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-1tra.xlsx
Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-2tra.xlsx
Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-3tra.xlsx
Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-4tra.xlsx
Datasets/yeast-1_vs_7-5-fold/yeast-1_vs_7-5-5tra.xlsx
Ada1  =>  led7digit-0-2-4-5-6-7-8-9_vs_1-5-fold => Datasets/led7digit-0-2-4-5-6-7-8-9_vs_1-5-fold/led7digit-0-2-4-5-6-7-8-9_vs_1-5-1tra.xlsx
Datasets/led7digit-0-2-4-5-6-7-8-9_vs_1-5-fold/led7digit-0-2-4-5-6-7-8-9_vs_1-5-2tra.xlsx
Datasets/led7digit-0-2-4-5-6-7-8-9_vs_1-5-fold/led7digi

In [168]:
troubs = {}
for dataset in datasets:
    for classifier in classifiers:
        files = glob.glob('./Results/'+dataset+'/'+classifier+'/*.npy')
        if len(files) != 6:
            if dataset in troubs:
                troubs[dataset].append(classifier)
            else:
                troubs[dataset] = [classifier]
            print(dataset, classifier)
        
# print([dataset for dataset in troubs if len(troubs[dataset])==4])

ecoli-0-2-6-7_vs_3-5-5-fold Ada1
shuttle-c2-vs-c4-5-fold SMOTE
yeast4-5-fold Ada1
abalone19-5- Ada1
abalone19-5- AdaCost
ecoli-0-1-3-7_vs_2-6-5-fold SMOTE
ecoli-0-1-3-7_vs_2-6-5-fold Ada1
pima-5-fold Ada1
abalone19-5-fold Ada1
abalone19-5-fold AdaCost
vowel0-5-fold SMOTE
led7digit-0-2-4-5-6-7-8-9_vs_1-5- SMOTE
ecoli-0-1_vs_2-3-5-5-fold SMOTE
glass-0-1-6_vs_2-5-fold SMOTE
yeast-1-2-8-9_vs_7-5-fold RUS
yeast-1-2-8-9_vs_7-5-fold SMOTE
ecoli-0-3-4_vs_5-5-fold SMOTE
new-thyroid2-5-fold SMOTE


In [143]:
# convert categorical variables into numerical
from sklearn.preprocessing import LabelEncoder
 
le = LabelEncoder()
 
# Using .fit_transform function to fit label
# encoder and return encoded label

base_path = 'Datasets/'
need_to_convert = ['abalone19-5-', 'abalone19-5-fold']
for needed in need_to_convert[1:]:
    for datFile in glob.glob(base_path+needed+'/*.dat'):
        print(datFile)
        df = read_dot_dat_file(datFile)
        label = le.fit_transform(df['Sex,'])        
        df['Sex,'] = label
        # df.to_excel(base_path+needed+'/'+datFile.split('/')[-1].split('.')[0]+'.xlsx')
        break
        

Datasets/abalone19-5-fold/abalone19-5-5tst.dat
['{positive,', 'negative}']


In [159]:
df = read_dot_dat_file(glob.glob(base_path+datasets[0]+'/*.dat')[0])
df.to_excel('temp.xlsx')

['{1,', '-1}']


In [167]:
pd.read_excel('temp.xlsx').iloc[:, 0:-1]

,Unnamed: 0,"Mcg,","Gvh,","Alm,","Mit,","Erl,","Vac,",Nuc
0,0,0.58,0.44,0.57,0.13,0.5,0.54,0.22
1,1,0.48,0.45,0.59,0.20,0.5,0.58,0.34
2,2,0.43,0.39,0.54,0.21,0.5,0.53,0.27
3,3,0.42,0.37,0.59,0.20,0.5,0.52,0.29
4,4,0.50,0.34,0.55,0.21,0.5,0.49,0.22
...,...,...,...,...,...,...,...,...
363,363,0.65,0.58,0.53,0.21,0.5,0.51,0.22
364,364,0.48,0.49,0.49,0.26,0.5,0.39,0.27
365,365,0.57,0.54,0.32,0.15,0.5,0.52,0.25
366,366,0.40,0.36,0.32,0.16,0.5,0.51,0.27


In [151]:
dataset = 'abalone19-5-'

traFiles = sorted(glob.glob(base_path+dataset+'/*tra.xlsx'))
tstFiles = sorted(glob.glob(base_path+dataset+'/*tst.xlsx'))
for traPath, tstPath in zip(traFiles, tstFiles):
    print(traPath)

    df_train = pd.read_excel(traPath)
    df_test = pd.read_excel(tstPath)

    x_train= df_train.iloc[:, 0:-1]
    y_train = df_train.iloc[:, -1]
    x_test= df_test.iloc[:, 0:-1]
    y_test = df_test.iloc[:, -1]
    break
df_train

Datasets/abalone19-5-/abalone19-5-1tra.xlsx


,Unnamed: 0,"Sex,","Length,","Diameter,","Height,","Whole_weight,","Shucked_weight,","Viscera_weight,",Shell_weight,Class
0,0,2,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,-1
1,1,2,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,-1
2,2,0,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,-1
3,3,2,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,-1
4,4,1,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,-1
...,...,...,...,...,...,...,...,...,...,...
3334,3334,2,0.520,0.385,0.165,0.7910,0.3750,0.1800,0.1815,-1
3335,3335,0,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,-1
3336,3336,2,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,-1
3337,3337,2,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,-1


In [142]:
df_train.reset_index()

,index,Unnamed: 0,"Sex,","Length,","Diameter,","Height,","Whole_weight,","Shucked_weight,","Viscera_weight,",Shell_weight,Class
0,0,0,2,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,-1
1,1,1,2,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,-1
2,2,2,0,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,-1
3,3,3,2,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,-1
4,4,4,1,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,-1
...,...,...,...,...,...,...,...,...,...,...,...
3334,3334,3334,2,0.520,0.385,0.165,0.7910,0.3750,0.1800,0.1815,-1
3335,3335,3335,0,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,-1
3336,3336,3336,2,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,-1
3337,3337,3337,2,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,-1
